In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

In [ ]:
file_path = "faults.xlsx"  
data = pd.read_excel(file_path)
X = data.iloc[:, :27].values
y = data.iloc[:, 27:].values

if y.shape[1] > 1:
    y_labels = np.argmax(y, axis=1)
else:
    le = LabelEncoder()
    y_labels = le.fit_transform(y.ravel())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(27,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_idx, val_idx in kf.split(X_scaled):
    X_train_cv, X_val_cv = X_scaled[train_idx], X_scaled[val_idx]
    y_train_cv, y_val_cv = y_labels[train_idx], y_labels[val_idx]
    
    cv_model = create_model()
    cv_model.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    loss, acc = cv_model.evaluate(X_val_cv, y_val_cv, verbose=0)
    cv_scores.append(acc)

print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

final_model = create_model()
final_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

y_pred_prob = final_model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision (Macro): {precision_score(y_test, y_pred, average='macro'):.4f}")
print(f"Recall (Macro): {recall_score(y_test, y_pred, average='macro'):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))